## PROGRAM extrct
Extracts data from the N-body output files Single.dat (fort.83) and Binary.dat (fort.82) relating to the overall cluster evolution, ie. half-mass radius and relaxation time, cluster mass, membership, core radius. 

Note: planets are ignored in this version. 

* Output in extrct.dat: 

*   Number of single stars + binaries 
*   Number of binaries 
*   Time (Myr) 
*   Relaxation time (Myr) 
*   Total cluster mass (Msun) 
*   Mass in core (Msun) 
*   Mass outside the tidal radius (Msun) 
*   Maximum stellar distance from cluster centre of mass (pc) 
*   Half-mass radius (pc) 
*   Radius containing inner 10% of cluster mass (pc) 
*   Core radius - as determined by Nbody code (pc) 
*   Number of systems (stars + binaries) inside the half-mass radius   
*   Number of systems within 1pc of the cluster centre    
*   Number of systems within the inner lagrangian radius (10%)  
*   Number of systems within the core radius 
*   Velocity dispersion (km/s)

In [2]:
#Necessary libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt

## Reformatting and reading data from fort.83 and fort.82
(Removing format of fort.83 and fort.82 - run this just once)

In [30]:
import os.path

file83 = os.path.isfile('fort.83-awk')
file82 = os.path.isfile('fort.82-awk')

if not file83:
    !awk '{print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12,$13}' fort.83 > fort.83-awk
    print "Removing formatting of fort.83"
if not file82:
    !awk '{print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12,$13,$14,$15,$16,$17,$18,1$9,$20,$21,$22,$23,$24}' fort.82 > fort.82-awk
    print "Removing formatting of fort.83"

In [14]:
N = 0
with open("fort.83-awk", 'r') as file1:
    with open("fort.82-awk", 'r') as file2:
        while file1 and file2:
            line11 = file1.readline()
            line21 = file2.readline()
            if not (line11 and line21):
                break
            words11 = line11.split()
            N = int(words11[0])
            #print N
            if N != -1000: 
                line12 = file1.readline()
                words12 = line12.split()
                line13 = file1.readline()
                words13 = line13.split()
                block = np.fromfile(file1, sep=' ', count=13*N).reshape([N,13])
            N = 0
            break

In [11]:
block[N-1,:]

array([  6.67000000e+02,   0.00000000e+00,   1.01000000e-01,
        -3.01100000e+00,  -8.72000000e-01,   7.63037000e-01,
        -9.95429100e-01,  -1.32544300e-01,  -4.60842900e-01,
         2.97174600e-01,   3.45059200e-01,   0.00000000e+00,
         3.79400000e+00])